In [1]:
#Import packages
#---------------------------------------
import sys
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
import warnings
import random
import arviz as az
import pymc as pm
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#Import your modules
#---------------------------------------
import admin_functions as adfn
import cell_decomp_func as cdfn


# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
%load_ext autoreload
sys.version

'3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]'

In [48]:
dataset = 'data1'
## Load simulated data from R
spots_st = adfn.return_files(s_data, 'spatial_transcriptomics/simulate_10x_spots/' + dataset + '/', '*simulated_spots*')[0]
props_st = adfn.return_files(s_data, 'spatial_transcriptomics/simulate_10x_spots/' + dataset + '/', '*true*')[0]
ref_st = adfn.return_files(s_data, 'spatial_transcriptomics/simulate_10x_spots/'  + dataset + '/', '*ref*')[0]
de_st = pd.read_csv(s_data + 'spatial_transcriptomics/simulate_10x_spots/de_genes.csv')
spots_st, props_st, ref_st

('simulated_spots.csv', 'true_weights.csv', 'reference.csv')

In [49]:
def load(spots_st, props_st, ref_st):
    spots = pd.read_csv(spots_st)
    prop_vec = pd.read_csv(props_st).T
    prop_vec = np.asarray(prop_vec.drop(prop_vec.index[0]))
    prop_vec = np.float64(prop_vec)
    ref_exp = pd.read_csv(ref_st)

    #Find DE genes
    degene_l = np.array(de_st['gene']) 
    ind=[]
    for de in degene_l:
        ind = np.append(ind, np.where(spots['gene'] == de)[0][0])

    #Keep only DE genes
    spots = spots.iloc[ind].T
    spots = np.asarray(spots.drop(spots.index[0]))
    spots = np.float64(spots)
    ref_exp = ref_exp.iloc[ind].T
    ref_exp = np.asarray(ref_exp.drop(ref_exp.index[0]))
    ref_exp = (ref_exp/np.reshape(np.sum(ref_exp, axis=1), (ref_exp.shape[0],1))) #convert to a rate
    ref_exp = np.float64(ref_exp)
    return(ref_exp, spots, prop_vec)

In [50]:
genes_list = np.linspace(20, 1000, 5).astype(int)
spots_list = np.linspace(20, 1000, 5).astype(int)
list_list = [genes_list, spots_list]
mode_list= ['ngenes', 'nspots']
lik_l = ['poisson', 'negbin']


In [ ]:
# Run alpha noise
import time

model_type = 'all' #CHANGE!

for x,li in enumerate(list_list):
    mode = mode_list[x]

    for l in li:
        #Set default parameters
        top_cells = 9
        top_genes = 500
        top_spots = 50
        if mode == 'nclusts':
            top_cells = l
        if mode == 'ngenes':
            top_genes = l
        if mode == 'nspots':
            top_spots=l

        ref_exp, spots, prop_vec = load(spots_st, props_st, ref_st)
        #+++++++++++++++++
        spots = spots[:top_spots,:top_genes]
        ref_exp = ref_exp[:top_cells, :top_genes]
        prop_vec = prop_vec[:top_spots,:top_cells]
        n_clusts = ref_exp.shape[0]
        n_spots = spots.shape[0]
        n_genes = spots.shape[1]
        #+++++++++++++++++

        # Run basic model
        start_time = time.time()
        #Define parameters of simulated data
        noise_type = 'gaussian' 

        for f in lik_l:
            model = cdfn.noise_pymc(n_clusts, n_spots, n_genes, ref_exp, spots, noise_type=noise_type, likelihood = f) #CHANGE!
            with model: data=pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)
            post, r2 = cdfn.model_stats(data, prop_vec, n_clusts)

            l = str(l)
            pref = l.zfill(4)
            fin = (time.time() - start_time)
            np.save(s_data + 'spatial_transcriptomics/RCTD-test_noise-' + noise_type + '_model-' + model_type + '_' + 'lik-' + f + '_10x-pbmc-' + dataset +  '_' + mode + '-' + pref + '.npy', np.array([prop_vec, post, r2, fin], dtype=object))
            print(l)
